In [29]:
import math
import pandas as pd
from numpy import *
from matplotlib.pyplot import * 
from mpl_toolkits.axes_grid1 import ImageGrid
from scipy.stats import skew
from lmfit.models import SkewedGaussianModel

rc('text', usetex=True, fontsize=20)

In [42]:
#Functions

def find_bifurcation_index(intensities):
    i_prev = 0
    bif_index = -1
    
    intensities_length = len(intensities)
    start_index = 0.3*intensities_length
    end_index = 0.7*intensities_length
    
    intensities = intensities[start_index:end_index]
    
    for i in range(len(intensities)-1):
        i_curr = intensities[i]
        i_next = intensities[i+1]
        if(i_prev >= i_curr and i_curr<=i_next):
            bif_index = i
        i_prev = i_curr
        
    if (bif_index==-1):
        return 0
    else:
        return bif_index+start_index

In [10]:
num_thetas = 9
cos_thetas = linspace(0,1,num_thetas+1) 
cos_thetas_plus = linspace(-1,0,num_thetas+1) 

thetas = sort(array([arccos(ct) for ct in cos_thetas]))
thetas_plus = array([arccos(ctp) for ctp in cos_thetas_plus])

In [11]:
print thetas
print thetas_plus
print rad2deg(thetas)
print rad2deg(thetas_plus)

[ 0.          0.47588225  0.67967382  0.84106867  0.98176536  1.11024234
  1.23095942  1.34670323  1.45945531  1.57079633]
[ 3.14159265  2.6657104   2.46191883  2.30052398  2.1598273   2.03135032
  1.91063324  1.79488942  1.68213734  1.57079633]
[  0.          27.26604445  38.94244127  48.1896851   56.2510114
  63.61220004  70.52877937  77.16041159  83.62062979  90.        ]
[ 180.          152.73395555  141.05755873  131.8103149   123.7489886
  116.38779996  109.47122063  102.83958841   96.37937021   90.        ]


In [12]:
### --------------- tau10E5 --------------- ###

vrots = [50,100]
vouts = [5,10,15,20]

all_x_frec = []
all_z_u = []
vrot_labs = []
vout_labs = []

for vout in vouts:
    for vrot in vrots:
        df = pd.read_csv('../../data/tau10E5/vrot'+str(vrot)+'/vout'+str(vout)+'/tau10E5_vrot'+str(vrot)+'_vout'+str(vout)+'_out.ascii', delimiter=' ')
        
        escaped = df['escaped']
        x_frec = df['x_frec']
        z_u = df['z_u']
        
        inds = where(escaped == 0)[0]
        
        x_frec_escaped = array(x_frec)[inds]
        z_u_escaped = array(z_u)[inds]
        
        all_x_frec.append(x_frec_escaped)
        all_z_u.append(z_u_escaped)
              
        vrot_labs.append(vrot)
        vout_labs.append(vout)

In [34]:
info_file = open('info_tau10E5.dat','w') 
info_file.write('vrot\tvout\ttheta_lower\ttheta_upper\tstd\tskw\tamplitude_neg\tsigma_neg\tcenter_neg\tgamma_neg\tamplitude_pos\tsigma_pos\tcenter_pos\tgamma_pos\n') # if skw > 0 there is more weight in the left tail of the distribution

for t in range(num_thetas):

    fig = figure(1, (16., 20.))
    grid = ImageGrid(fig, 111, # similar to subplot(111)
                    nrows_ncols = (4, 2), # creates 3x2 grid of axes
                    axes_pad=0.3, # pad between axes in inch.
                    aspect=False,)

    props = dict(boxstyle='square', facecolor='white')

    theta_lower = thetas[t]
    theta_upper = thetas[t+1]
    
    theta_plus_lower = thetas_plus[t+1]
    theta_plus_upper = thetas_plus[t]
    
    for i in range(8):
        
        current_z_u = all_z_u[i]
        current_x_frec = all_x_frec[i]
        
        acos_current_z_u = array([arccos(czu) for czu in current_z_u])
        
        angle_indices = where( ((acos_current_z_u >= theta_lower) & (acos_current_z_u < theta_upper)) | ((acos_current_z_u >= theta_plus_lower) & (acos_current_z_u < theta_plus_upper)))[0]
        
        current_x = current_x_frec[angle_indices] #only the ones between those upper and lower angles
        
        vrot_lab = r'${0:.0f}$'.format(vrot_labs[i])
        vout_lab = r'${0:.0f}$'.format(vout_labs[i])
        theta_lower_lab = r'${0:.0f}$'.format(int(rad2deg(theta_lower)))
        theta_upper_lab = r'${0:.0f}$'.format(int(rad2deg(theta_upper)))

        theta_lab = r'$\theta=$ '
        lab = '$v_{rot}=$ '+vrot_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$ \n $v_{out}=$ '+vout_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$ \n'+theta_lab+theta_lower_lab+'$^\circ$ $-$ '+theta_upper_lab+'$^\circ$'
        
        grid_i = grid[i]
        
        n, b = histogram(current_x, bins=30)        
        delta_x = b[1]-b[0]
        n = n/delta_x
        
        border_x = []
        border_y = []
        
        for j in range(len(n)-1):
            border_x.append(b[j+1])
            border_x.append(b[j+1])
            border_y.append(n[j])
            border_y.append(n[j+1])
        
        bifurcation_index = find_bifurcation_index(n)
        bifurcation = b[bifurcation_index]

        x_x_neg = b[0:bifurcation_index+1]
        y_x_neg = n[0:bifurcation_index+1]
        x_x_pos = b[bifurcation_index:-2]
        y_x_pos = n[bifurcation_index:-1]
        
        model_neg = SkewedGaussianModel()
        params_neg = model_neg.make_params(amplitude=max(x_x_neg), center = mean(x_x_neg), sigma=std(x_x_neg), gamma = -2)
        result_neg = model_neg.fit(y_x_neg, params_neg, x=x_x_neg)

        amplitude_neg = result_neg.best_values['amplitude']
        sigma_neg = result_neg.best_values['sigma']
        center_neg = result_neg.best_values['center']
        gamma_neg = result_neg.best_values['gamma']

        model_pos = SkewedGaussianModel()
        params_pos = model_pos.make_params(amplitude=max(x_x_pos), center = mean(x_x_pos), sigma=std(x_x_pos), gamma = 2)
        result_pos = model_pos.fit(y_x_pos, params_pos, x=x_x_pos)

        amplitude_pos = result_pos.best_values['amplitude']
        sigma_pos = result_pos.best_values['sigma']
        center_pos = result_pos.best_values['center']
        gamma_pos = result_pos.best_values['gamma']

        
        grid_i.bar(b[0:30], n, width=b[1]-b[0], color = 'c', edgecolor = 'c')
        #grid_i.bar(x_x_neg, y_x_neg, width=b[1]-b[0], color = 'c', edgecolor = 'c')
        #grid_i.bar(x_x_pos, y_x_pos, width=b[1]-b[0], color = 'b', edgecolor = 'b')
        #grid_i.plot(x_x_neg, result_neg.best_fit, c='k', linestyle='--')
        #grid_i.plot(x_x_pos, result_pos.best_fit, c='k', linestyle='--')
        grid_i.plot(border_x, border_y, c='k')
        #grid_i.vlines(bifurcation, 0, max(border_y)*1.5, color='b')

        grid_i.axvline(x=0, ymin=0, ymax=1800, c='k', linestyle='--')
        grid_i.set_xlim(-30,20)
        grid_i.set_ylim(0,1700)
        grid_i.set_ylabel('$\mathrm{Intensity}$')
        grid_i.set_xlabel('$\mathrm{x}$')
        grid_i.text(-28,1250, lab, fontsize=20, bbox=props)
        
        info_file.write(str(vrot_labs[i])+'\t'+str(vout_labs[i])+'\t'+str(theta_lower)+'\t'+str(theta_upper)+'\t'+str(std(current_x))+'\t'+str(skew(current_x))+str(amplitude_neg)+'\t'+str(sigma_neg)+'\t'+str(center_neg)+'\t'+str(gamma_neg)+'\t'+str(amplitude_pos)+'\t'+str(sigma_pos)+'\t'+str(center_pos)+'\t'+str(gamma_pos)+'\n')

    savefig('./tau10E5/tau10E5_phi'+str(int(rad2deg(theta_lower)))+'-'+str(int(rad2deg(theta_upper)))+'.png', format='png', transparent=False, bbox_inches='tight')
    close()
    
info_file.close()

In [38]:
### --------------- tau10E6 --------------- ###

vrots = [50,100]
vouts = [5,10,15,20]

all_x_frec = []
all_z_u = []
vrot_labs = []
vout_labs = []

for vout in vouts:
    for vrot in vrots:
        df = pd.read_csv('../../data/tau10E6/vrot'+str(vrot)+'/vout'+str(vout)+'/tau10E6_vrot'+str(vrot)+'_vout'+str(vout)+'_out.ascii', delimiter=' ')
        
        escaped = df['escaped']
        x_frec = df['x_frec']
        z_u = df['z_u']
        
        inds = where(escaped == 0)[0]
        
        x_frec_escaped = array(x_frec)[inds]
        z_u_escaped = array(z_u)[inds]
        
        all_x_frec.append(x_frec_escaped)
        all_z_u.append(z_u_escaped)
              
        vrot_labs.append(vrot)
        vout_labs.append(vout)

In [39]:
info_file = open('info_tau10E6.dat','w') 
info_file.write('vrot\tvout\ttheta_lower\ttheta_upper\tstd\tskw\tamplitude_neg\tsigma_neg\tcenter_neg\tgamma_neg\tamplitude_pos\tsigma_pos\tcenter_pos\tgamma_pos\n') # if skw > 0 there is more weight in the left tail of the distribution

for t in range(num_thetas):

    fig = figure(1, (16., 20.))
    grid = ImageGrid(fig, 111, # similar to subplot(111)
                    nrows_ncols = (4, 2), # creates 3x2 grid of axes
                    axes_pad=0.3, # pad between axes in inch.
                    aspect=False,)

    props = dict(boxstyle='square', facecolor='white')

    theta_lower = thetas[t]
    theta_upper = thetas[t+1]
    
    theta_plus_lower = thetas_plus[t+1]
    theta_plus_upper = thetas_plus[t]
    
    for i in range(8):
        
        current_z_u = all_z_u[i]
        current_x_frec = all_x_frec[i]
        
        acos_current_z_u = array([arccos(czu) for czu in current_z_u])
        
        angle_indices = where( ((acos_current_z_u >= theta_lower) & (acos_current_z_u < theta_upper)) | ((acos_current_z_u >= theta_plus_lower) & (acos_current_z_u < theta_plus_upper)))[0]
        
        current_x = current_x_frec[angle_indices] #only the ones between those upper and lower angles
        
        vrot_lab = r'${0:.0f}$'.format(vrot_labs[i])
        vout_lab = r'${0:.0f}$'.format(vout_labs[i])
        theta_lower_lab = r'${0:.0f}$'.format(int(rad2deg(theta_lower)))
        theta_upper_lab = r'${0:.0f}$'.format(int(rad2deg(theta_upper)))

        theta_lab = r'$\theta=$ '
        lab = '$v_{rot}=$ '+vrot_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$ \n $v_{out}=$ '+vout_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$ \n'+theta_lab+theta_lower_lab+'$^\circ$ $-$ '+theta_upper_lab+'$^\circ$'
        
        grid_i = grid[i]
        
        n, b = histogram(current_x, bins=30)        
        delta_x = b[1]-b[0]
        n = n/delta_x
        
        border_x = []
        border_y = []
        
        for j in range(len(n)-1):
            border_x.append(b[j+1])
            border_x.append(b[j+1])
            border_y.append(n[j])
            border_y.append(n[j+1])
        
        bifurcation_index = find_bifurcation_index(n)
        bifurcation = b[bifurcation_index]

        x_x_neg = b[0:bifurcation_index+1]
        y_x_neg = n[0:bifurcation_index+1]
        x_x_pos = b[bifurcation_index:-2]
        y_x_pos = n[bifurcation_index:-1]
        
        model_neg = SkewedGaussianModel()
        params_neg = model_neg.make_params(amplitude=max(x_x_neg), center = mean(x_x_neg), sigma=std(x_x_neg), gamma = -2)
        result_neg = model_neg.fit(y_x_neg, params_neg, x=x_x_neg)

        amplitude_neg = result_neg.best_values['amplitude']
        sigma_neg = result_neg.best_values['sigma']
        center_neg = result_neg.best_values['center']
        gamma_neg = result_neg.best_values['gamma']

        model_pos = SkewedGaussianModel()
        params_pos = model_pos.make_params(amplitude=max(x_x_pos), center = mean(x_x_pos), sigma=std(x_x_pos), gamma = 2)
        result_pos = model_pos.fit(y_x_pos, params_pos, x=x_x_pos)

        amplitude_pos = result_pos.best_values['amplitude']
        sigma_pos = result_pos.best_values['sigma']
        center_pos = result_pos.best_values['center']
        gamma_pos = result_pos.best_values['gamma']

        
        grid_i.bar(b[0:30], n, width=b[1]-b[0], color = 'c', edgecolor = 'c')
        #grid_i.bar(x_x_neg, y_x_neg, width=b[1]-b[0], color = 'c', edgecolor = 'c')
        #grid_i.bar(x_x_pos, y_x_pos, width=b[1]-b[0], color = 'b', edgecolor = 'b')
        #grid_i.plot(x_x_neg, result_neg.best_fit, c='k', linestyle='--')
        #grid_i.plot(x_x_pos, result_pos.best_fit, c='k', linestyle='--')
        grid_i.plot(border_x, border_y, c='k')
        #grid_i.vlines(bifurcation, 0, max(border_y)*1.5, color='b')

        grid_i.axvline(x=0, ymin=0, ymax=1800, c='k', linestyle='--')
        grid_i.set_xlim(-30,20)
        grid_i.set_ylim(0,1700)
        grid_i.set_ylabel('$\mathrm{Intensity}$')
        grid_i.set_xlabel('$\mathrm{x}$')
        grid_i.text(-28,1250, lab, fontsize=20, bbox=props)
        
        info_file.write(str(vrot_labs[i])+'\t'+str(vout_labs[i])+'\t'+str(theta_lower)+'\t'+str(theta_upper)+'\t'+str(std(current_x))+'\t'+str(skew(current_x))+str(amplitude_neg)+'\t'+str(sigma_neg)+'\t'+str(center_neg)+'\t'+str(gamma_neg)+'\t'+str(amplitude_pos)+'\t'+str(sigma_pos)+'\t'+str(center_pos)+'\t'+str(gamma_pos)+'\n')

    savefig('./tau10E6/tau10E6_phi'+str(int(rad2deg(theta_lower)))+'-'+str(int(rad2deg(theta_upper)))+'.png', format='png', transparent=False, bbox_inches='tight')
    close()
    
info_file.close()

In [40]:
### --------------- tau10E7 --------------- ###

vrots = [50,100]
vouts = [5,10,15,20]

all_x_frec = []
all_z_u = []
vrot_labs = []
vout_labs = []

for vout in vouts:
    for vrot in vrots:
        df = pd.read_csv('../../data/tau10E7/vrot'+str(vrot)+'/vout'+str(vout)+'/tau10E7_vrot'+str(vrot)+'_vout'+str(vout)+'_out.ascii', delimiter=' ')
        
        escaped = df['escaped']
        x_frec = df['x_frec']
        z_u = df['z_u']
        
        inds = where(escaped == 0)[0]
        
        x_frec_escaped = array(x_frec)[inds]
        z_u_escaped = array(z_u)[inds]
        
        all_x_frec.append(x_frec_escaped)
        all_z_u.append(z_u_escaped)
              
        vrot_labs.append(vrot)
        vout_labs.append(vout)

In [47]:
info_file = open('info_tau10E7.dat','w') 
info_file.write('vrot\tvout\ttheta_lower\ttheta_upper\tstd\tskw\tamplitude_neg\tsigma_neg\tcenter_neg\tgamma_neg\tamplitude_pos\tsigma_pos\tcenter_pos\tgamma_pos\n') # if skw > 0 there is more weight in the left tail of the distribution

for t in range(num_thetas):

    fig = figure(1, (16., 20.))
    grid = ImageGrid(fig, 111, # similar to subplot(111)
                    nrows_ncols = (4, 2), # creates 3x2 grid of axes
                    axes_pad=0.3, # pad between axes in inch.
                    aspect=False,)

    props = dict(boxstyle='square', facecolor='white')

    theta_lower = thetas[t]
    theta_upper = thetas[t+1]
    
    theta_plus_lower = thetas_plus[t+1]
    theta_plus_upper = thetas_plus[t]
    
    for i in range(8):
        
        current_z_u = all_z_u[i]
        current_x_frec = all_x_frec[i]
        
        acos_current_z_u = array([arccos(czu) for czu in current_z_u])
        
        angle_indices = where( ((acos_current_z_u >= theta_lower) & (acos_current_z_u < theta_upper)) | ((acos_current_z_u >= theta_plus_lower) & (acos_current_z_u < theta_plus_upper)))[0]
        
        current_x = current_x_frec[angle_indices] #only the ones between those upper and lower angles
        
        vrot_lab = r'${0:.0f}$'.format(vrot_labs[i])
        vout_lab = r'${0:.0f}$'.format(vout_labs[i])
        theta_lower_lab = r'${0:.0f}$'.format(int(rad2deg(theta_lower)))
        theta_upper_lab = r'${0:.0f}$'.format(int(rad2deg(theta_upper)))

        theta_lab = r'$\theta=$ '
        lab = '$v_{rot}=$ '+vrot_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$ \n $v_{out}=$ '+vout_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$ \n'+theta_lab+theta_lower_lab+'$^\circ$ $-$ '+theta_upper_lab+'$^\circ$'
        
        grid_i = grid[i]
        
        n, b = histogram(current_x, bins=30)        
        delta_x = b[1]-b[0]
        n = n/delta_x
        
        border_x = []
        border_y = []
        
        for j in range(len(n)-1):
            border_x.append(b[j+1])
            border_x.append(b[j+1])
            border_y.append(n[j])
            border_y.append(n[j+1])
        
        bifurcation_index = find_bifurcation_index(n)
        bifurcation = b[bifurcation_index]

        x_x_neg = b[0:bifurcation_index+1]
        y_x_neg = n[0:bifurcation_index+1]
        x_x_pos = b[bifurcation_index:-2]
        y_x_pos = n[bifurcation_index:-1]
        
        model_neg = SkewedGaussianModel()
        params_neg = model_neg.make_params(amplitude=max(x_x_neg), center = mean(x_x_neg), sigma=std(x_x_neg), gamma = -2)
        result_neg = model_neg.fit(y_x_neg, params_neg, x=x_x_neg)

        amplitude_neg = result_neg.best_values['amplitude']
        sigma_neg = result_neg.best_values['sigma']
        center_neg = result_neg.best_values['center']
        gamma_neg = result_neg.best_values['gamma']

        model_pos = SkewedGaussianModel()
        params_pos = model_pos.make_params(amplitude=max(x_x_pos), center = mean(x_x_pos), sigma=std(x_x_pos), gamma = 2)
        result_pos = model_pos.fit(y_x_pos, params_pos, x=x_x_pos)

        amplitude_pos = result_pos.best_values['amplitude']
        sigma_pos = result_pos.best_values['sigma']
        center_pos = result_pos.best_values['center']
        gamma_pos = result_pos.best_values['gamma']

        
        grid_i.bar(b[0:30], n, width=b[1]-b[0], color = 'c', edgecolor = 'c')
        #grid_i.bar(x_x_neg, y_x_neg, width=b[1]-b[0], color = 'c', edgecolor = 'c')
        #grid_i.bar(x_x_pos, y_x_pos, width=b[1]-b[0], color = 'b', edgecolor = 'b')
        #grid_i.plot(x_x_neg, y_x_neg, c='y') ##
        #grid_i.plot(x_x_pos, y_x_pos, c='y') ##
        #grid_i.plot(x_x_neg, result_neg.best_fit, c='k', linestyle='--')
        #grid_i.plot(x_x_pos, result_pos.best_fit, c='k', linestyle='--')
        grid_i.plot(border_x, border_y, c='k')
        #grid_i.vlines(bifurcation, 0, max(border_y)*1.5, color='b')

        grid_i.axvline(x=0, ymin=0, ymax=1800, c='k', linestyle='--')
        grid_i.set_xlim(-30,20)
        grid_i.set_ylim(0,1700)
        grid_i.set_ylabel('$\mathrm{Intensity}$')
        grid_i.set_xlabel('$\mathrm{x}$')
        grid_i.text(-28,1250, lab, fontsize=20, bbox=props)
        
        info_file.write(str(vrot_labs[i])+'\t'+str(vout_labs[i])+'\t'+str(theta_lower)+'\t'+str(theta_upper)+'\t'+str(std(current_x))+'\t'+str(skew(current_x))+str(amplitude_neg)+'\t'+str(sigma_neg)+'\t'+str(center_neg)+'\t'+str(gamma_neg)+'\t'+str(amplitude_pos)+'\t'+str(sigma_pos)+'\t'+str(center_pos)+'\t'+str(gamma_pos)+'\n')

    savefig('./tau10E7/tau10E7_phi'+str(int(rad2deg(theta_lower)))+'-'+str(int(rad2deg(theta_upper)))+'.png', format='png', transparent=False, bbox_inches='tight')
    close()
    
info_file.close()